In [1]:
#### url:https://www.redbus.in/online-booking/rsrtc/?utm_source=rtchometile
### webscraping_bus route details in red bus:Rajasthan

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import time
import pandas as pd

# Initialize the WebDriver and open the website
driver = webdriver.Chrome()
driver.maximize_window()
driver.get('https://www.redbus.in/online-booking/rsrtc/?utm_source=rtchometile')

# List to store the collected bus route data
bus_route_data = []

# Function to collect route details from the page
def route_details():
    try:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "route_link")))
        route_elements = driver.find_elements(By.CLASS_NAME, "route")
        
        route_links = [route.get_attribute("href") for route in route_elements]
        route_names = [route.get_attribute("title").strip() for route in route_elements]
        
        # Collect the route details for this page and return them
        return [{'route_name': name, 'route_link': link} for name, link in zip(route_names, route_links)]
        
    except Exception as e:
        print(f"An error occurred: {e}")
        return []

# Function to navigate through all the pages and scrape route details
def scrape_all_pages():
    all_routes = []
    try:
        # Loop through the first 5 pages (or adjust the range as needed)
        for page in range(1, 6):  # Assuming there are 5 pages
            if page > 1:
                # Wait for the pagination tab of the current page to be clickable
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                # Scroll to the pagination tab and click it
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the new page to load
            
            # Collect the route details after navigating to the page (either the first page or after pagination)
            all_routes.extend(route_details())

    except Exception as e:
        print(f"Error occurred while accessing page {page}: {str(e)}")
    return all_routes

# Function to scrape bus details for each route
def scrape_bus_details(route_name, route_link):
    try:
        driver.get(route_link)
        time.sleep(5)

        # Try to click the "View Buses" button if available
        try:
            view_buses = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses)
            time.sleep(5)
        except:
            print(f"No 'View Buses' button found for {route_link}")

        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)

        # Extract data from the page
        bus_names = driver.find_elements(By.CSS_SELECTOR, "div.travels.lh-24.f-bold.d-color")
        bus_types = driver.find_elements(By.CSS_SELECTOR, "div.bus-type.f-12.m-top-16.l-color.evBus")
        departure_times = driver.find_elements(By.CSS_SELECTOR, "div.dp-time.f-19.d-color.f-bold")
        durations = driver.find_elements(By.CSS_SELECTOR, "div.dur.l-color.lh-24")
        reaching_times = driver.find_elements(By.CSS_SELECTOR, "div.bp-time.f-19.d-color.disp-Inline")
        star_ratings = driver.find_elements(By.CSS_SELECTOR, "div.lh-18.rating.rat-red")
        prices = driver.find_elements(By.CSS_SELECTOR, "div.fare.d-block")
        seat_availabilities = driver.find_elements(By.CSS_SELECTOR, "div.seat-left.m-top-16")
    
        bus_route_details = []
    
        # Loop through each bus and store its details
        for i in range(len(bus_names)):
            bus_detail = {
                "State": "Rajasthan",
                "Route Name": route_name,
                "Route Link": route_link,
                "Bus Name": bus_names[i].text,
                "Bus Type": bus_types[i].text if i < len(bus_types) else "N/A",
                "Departure Time": departure_times[i].text if i < len(departure_times) else "N/A",
                "Duration": durations[i].text if i < len(durations) else "N/A",
                "Reaching Time": reaching_times[i].text if i < len(reaching_times) else "N/A",
                "Rating": star_ratings[i].text if i < len(star_ratings) else "N/A",
                "Price": prices[i].text if i < len(prices) else "N/A",
                "Seats Available": seat_availabilities[i].text if i < len(seat_availabilities) else "N/A"
            }
            bus_route_details.append(bus_detail)
                     
        return bus_route_details
    except Exception as e:
        print(f"Error occurred while accessing {route_link}: {str(e)}")
        return []   

# Scrape all route details
routes = scrape_all_pages()

# For each route, scrape bus details and add to the bus_route_data list
for route in routes:
    bus_details = scrape_bus_details(route["route_name"], route["route_link"])
    if bus_details:
        bus_route_data.extend(bus_details)  # Add bus details to the bus_route_data list

# Convert bus_route_data to a pandas DataFrame
df = pd.DataFrame(bus_route_data)

# Save the DataFrame to a CSV file
df.to_csv('Rajasthan_bus_details.csv', index=False)

# Optionally, print the collected bus details
print("Data saved to Rajasthan_bus_details.csv")

# Close the driver after scraping
driver.quit()


Error occurred while accessing page 3: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7F0C96CF5+28821]
	(No symbol) [0x00007FF7F0C03880]
	(No symbol) [0x00007FF7F0AA578A]
	(No symbol) [0x00007FF7F0AF91BE]
	(No symbol) [0x00007FF7F0AF94AC]
	(No symbol) [0x00007FF7F0B42647]
	(No symbol) [0x00007FF7F0B1F33F]
	(No symbol) [0x00007FF7F0B3F412]
	(No symbol) [0x00007FF7F0B1F0A3]
	(No symbol) [0x00007FF7F0AEA778]
	(No symbol) [0x00007FF7F0AEB8E1]
	GetHandleVerifier [0x00007FF7F0FCFCED+3408013]
	GetHandleVerifier [0x00007FF7F0FE745F+3504127]
	GetHandleVerifier [0x00007FF7F0FDB63D+3455453]
	GetHandleVerifier [0x00007FF7F0D5BDFB+835995]
	(No symbol) [0x00007FF7F0C0EB9F]
	(No symbol) [0x00007FF7F0C0A854]
	(No symbol) [0x00007FF7F0C0A9ED]
	(No symbol) [0x00007FF7F0BFA1D9]
	BaseThreadInitThunk [0x00007FF8B0EFDBE7+23]
	RtlUserThreadStart [0x00007FF8B1E7FBEC+44]

Data saved to Rajasthan_bus_details.csv


In [3]:
df

,State,Route Name,Route Link,Bus Name,Bus Type,Departure Time,Duration,Reaching Time,Rating,Price,Seats Available
0,Rajasthan,Jodhpur to Ajmer,https://www.redbus.in/bus-tickets/jodhpur-to-a...,RSRTC - 160785,Super Luxury Volvo AC Seater Pushback 2+2,17:30,04h 00m,21:30,2.0,INR 460,31 Seats available
1,Rajasthan,Jodhpur to Ajmer,https://www.redbus.in/bus-tickets/jodhpur-to-a...,RSRTC - 190951,Express Non AC Seater 2+3,18:00,04h 30m,22:30,2.8,INR 229,42 Seats available
2,Rajasthan,Jodhpur to Ajmer,https://www.redbus.in/bus-tickets/jodhpur-to-a...,RSRTC - 193481,Express Non AC Seater 2+3,18:30,04h 30m,23:00,2.6,INR 229,42 Seats available
3,Rajasthan,Jodhpur to Ajmer,https://www.redbus.in/bus-tickets/jodhpur-to-a...,RSRTC - 176394,Super Luxury Volvo AC Seater Pushback 2+2,20:00,03h 45m,23:45,2.8,INR 460,37 Seats available
4,Rajasthan,Jodhpur to Ajmer,https://www.redbus.in/bus-tickets/jodhpur-to-a...,RSRTC - 189473,NON AC Seater / Sleeper 2+1,21:00,04h 30m,01:30,2.1,INR 302,16 Seats available
...,...,...,...,...,...,...,...,...,...,...,...
710,Rajasthan,Mathura to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/mathura-to-j...,Rinku Travels,A/C Seater / Sleeper (2+1),23:35,06h 40m,06:15,1.6,INR 600,N/A
711,Rajasthan,Mathura to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/mathura-to-j...,GV Tours,A/C Seater / Sleeper (2+1),21:15,06h 45m,04:00,N/A,463,N/A
712,Rajasthan,Mathura to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/mathura-to-j...,Bhagwati Travel Agency,A/C Seater / Sleeper (2+1),22:40,06h 00m,04:40,N/A,449,N/A
713,Rajasthan,Mathura to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/mathura-to-j...,Amar Travels (Agra),NON A/C Sleeper (2+1),16:15,07h 15m,23:30,N/A,1620,N/A
